## Implementation

In [24]:
from IOHMM import IOHMM_model
import numpy as np, pandas as pd
import torch

In [25]:
!pip install pandas

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python3.9 -m pip install --upgrade pip


In [26]:
data = pd.read_csv("data/Financial-Data/stocks/TSLA.csv")
data.head()
data = data.dropna()

input = torch.tensor(np.array(data[['Open']]), dtype=torch.float32)
output = torch.tensor(np.array(data['Close']), dtype=torch.float32)

# Example 1

## Set up a simple model manully

In [27]:
IOHMM = IOHMM_model(num_states=2, inputs=input[:10], outputs=output[:10], max_iter=1000, tol=1e-6)
print(IOHMM.initial_pi)
print(IOHMM.transition_matrix)
print(IOHMM.emission_matrix)
print(IOHMM.sd)

Parameter containing:
tensor([0.5000, 0.5000], requires_grad=True)
Parameter containing:
tensor([[[-0.1955, -0.9656],
         [ 0.4224,  0.2673]],

        [[-0.4212, -0.5107],
         [-1.5727, -0.1232]]], requires_grad=True)
Parameter containing:
tensor([[ 3.5870, -1.8313],
        [ 1.5987, -1.2770]], requires_grad=True)
Parameter containing:
tensor([1., 1.], requires_grad=True)


In [28]:
print(IOHMM._forward())
torch.sum(IOHMM._forward(), dim=1)

tensor([[0.7765, 0.2235],
        [0.5615, 0.4385],
        [0.6802, 0.3198],
        [0.5359, 0.4641],
        [0.5223, 0.4777],
        [0.4397, 0.5603],
        [0.5180, 0.4820],
        [0.5079, 0.4921],
        [0.5147, 0.4853],
        [0.5219, 0.4781]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [29]:
print(IOHMM._backward())
torch.sum(IOHMM._backward(), dim=1)

tensor([[0.5867, 0.4133],
        [0.6622, 0.3378],
        [0.6457, 0.3543],
        [0.6187, 0.3813],
        [0.5558, 0.4442],
        [0.4797, 0.5203],
        [0.4519, 0.5481],
        [0.5551, 0.4449],
        [0.4357, 0.5643],
        [0.6371, 0.3629]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [30]:
print(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()))
torch.sum(IOHMM._compute_gamma(IOHMM._forward(), IOHMM._backward()), dim=1)

tensor([[0.8314, 0.1686],
        [0.7152, 0.2848],
        [0.7949, 0.2051],
        [0.6520, 0.3480],
        [0.5777, 0.4223],
        [0.4199, 0.5801],
        [0.4698, 0.5302],
        [0.5629, 0.4371],
        [0.4502, 0.5498],
        [0.6571, 0.3429]])


tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])

In [31]:
print(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()))
a = torch.sum(IOHMM._compute_xi(IOHMM._forward(), IOHMM._backward()), axis=1)
torch.sum(a, axis=1)

tensor([[[0.0652, 0.3877],
         [0.4061, 0.1411]],

        [[0.0785, 0.2302],
         [0.6190, 0.0723]],

        [[0.0527, 0.3973],
         [0.4186, 0.1314]],

        [[0.0717, 0.2849],
         [0.5429, 0.1005]],

        [[0.0581, 0.3486],
         [0.4457, 0.1476]],

        [[0.0610, 0.3062],
         [0.4724, 0.1604]],

        [[0.0491, 0.3379],
         [0.4164, 0.1966]],

        [[0.0666, 0.3670],
         [0.4200, 0.1464]],

        [[0.0472, 0.2770],
         [0.4954, 0.1804]],

        [[0.0801, 0.4417],
         [0.3535, 0.1247]]])


tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [32]:
IOHMM._baum_welch()

## See the training results

In [33]:
print("Initial pi")
print(IOHMM.initial_pi) # to normalize
print("Transition matrix")
print(IOHMM.transition_matrix)
print("Emission matrix")
print(IOHMM.emission_matrix)
print("Sd")
print(IOHMM.sd)

Initial pi
Parameter containing:
tensor([9640185.0000, 2007872.8750], requires_grad=True)
Transition matrix
Parameter containing:
tensor([[[-0.1955, -0.9656],
         [ 0.4224,  0.2673]],

        [[-0.4212, -0.5107],
         [-1.5727, -0.1232]]], requires_grad=True)
Emission matrix
Parameter containing:
tensor([[ 3.5870, -1.8313],
        [ 1.5987, -1.2770]], requires_grad=True)
Sd
Parameter containing:
tensor([1., 1.], requires_grad=True)


## Viterbi

In [34]:
#call the viterbi algorithm
IOHMM.viterbi()